In [1]:
!pip install mlflow boto3 awscli optuna xgboost imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 126.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [3]:
import mlflow
#Setup MLflow tracking server
mlflow.set_tracking_uri("http://18.117.193.162:5000/")

#set experiment
mlflow.set_experiment("Exp 5- Machine Learning Algorithms with HP tuning")

2025/08/21 18:29:52 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5- Machine Learning Algorithms with HP tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://campusxalidvcbucket/425591744193892905', creation_time=1755800992412, experiment_id='425591744193892905', last_update_time=1755800992412, lifecycle_stage='active', name='Exp 5- Machine Learning Algorithms with HP tuning', tags={}>

In [4]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [5]:
df = pd.read_csv('reddit_preprocessing.csv')
df.shape

(36793, 2)

In [9]:
#Step1: Remap the class labels from [-1, 0, 1] to [2,0,1]
df.loc[:, 'category'] = df['category'].map({-1:2, 0:0, 1:1})

#Step2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Fill NaN values in 'clean_comment' with empty strings
df.loc[:, 'clean_comment'] = df['clean_comment'].fillna('')

ngram_range = (1,3)
max_features = 1000

#Step3: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

vectorizer = TfidfVectorizer(ngram_range = ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

smote = SMOTE(random_state = 42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

#Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
  with mlflow.start_run():
     # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


#Step5: Optuna Objective function for XGBoost
def objective_xgboost(trial):
  n_estimators = trial.suggest_int('n_estimators', 50, 300)
  learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
  max_depth = trial.suggest_int('max_depth', 3, 10)

  model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth = max_depth, random_state=42)
  return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


#Step7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
  study = optuna.create_study(direction = 'maximize')
  study.optimize(objective_xgboost, n_trials = 30)

  #Get the best parameters and log only the best model
  best_params = study.best_params
  best_model = XGBClassifier(n_estimators = best_params['n_estimators'], learning_rate = best_params['learning_rate'], max_depth = best_params['max_depth'], random_state=42)

  #log the model
  log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

run_optuna_experiment()

/tmp/ipython-input-2334878799.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_comment'] = df['clean_comment'].fillna('')
[I 2025-08-21 18:46:01,427] A new study created in memory with name: no-name-2ca71c52-a72b-4b71-86f6-4901150a7001
[I 2025-08-21 18:46:15,682] Trial 0 finished with value: 0.836398668768611 and parameters: {'n_estimators': 52, 'learning_rate': 0.08038885524793873, 'max_depth': 10}. Best is trial 0 with value: 0.836398668768611.
[I 2025-08-21 18:47:22,368] Trial 1 finished with value: 0.7143107374321247 and parameters: {'n_estimators': 221, 'learning_rate': 0.00039901790802863627, 'max_depth': 10}. Best is trial 0 with value: 0.836398668768611.
[I 2025-08-21 18:47:51,901] Trial 2 finished with value: 0.8994569977228

🏃 View run XGBoost_SMOTE_TFIDF_Trigrams at: http://18.117.193.162:5000/#/experiments/425591744193892905/runs/3b79b45999c84232aea008ea4b9f83f4
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/425591744193892905
